### Creation of the dataframe

Importing libraries

In [1]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

print('Libraries imported.')

Libraries imported.


In [2]:
# Creation the dataframe through the wiki link
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df_toronto = pd.read_html(link, header=0)[0]

In [3]:
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
# Remove Not assigned borough and reindexing
df_toronto = df_toronto[df_toronto.Borough != "Not assigned"].reset_index(drop=True)

# Group by the same postal code and borough transforming to df and after resetting index
df_toronto_g = df_toronto.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda tags: ', '.join(tags)).to_frame().reset_index()

In [5]:
# Before changing value of Neighbourhood
df_toronto_g[df_toronto_g.Postcode == "M7A"]

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


In [6]:
# if neighbourhood is "Not assigned" assign the value of Borough
df_toronto_g['Neighbourhood'] = np.where(df_toronto_g['Neighbourhood'] == 'Not assigned', df_toronto_g['Borough'], df_toronto_g['Neighbourhood'])

In [7]:
# After changing value of Neighbourhood
df_toronto_g[df_toronto_g.Postcode == "M7A"]

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [8]:
#Number of columns
df_toronto_g.shape

(103, 3)

## Second task 

Installing geocoder

In [9]:
import sys
!{sys.executable} -m pip install geocoder

In [11]:
#importing csv file with coordinates
!wget -O Geospatial_Coordinates.csv https://cocl.us/Geospatial_data

--2019-07-11 18:05:08--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-07-11 18:05:08--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197, 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-07-11 18:05:08--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [followin

In [13]:
# create df with geospatial coordinates
df_coordinates = pd.read_csv("Geospatial_Coordinates.csv")

In [15]:
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
# Adding two new columns
df_toronto_g['Latitude'] = None
df_toronto_g['Longitude'] = None

In [33]:
# For each Postal Code of the dataframe df_toronto add Latitude and Longitude (from df_coordinates)
for i in range(len(df_toronto_g)):
    post_code = df_toronto_g.Postcode[i]
    df_toronto_g.Latitude[i] = float(df_coordinates[df_coordinates['Postal Code'] == post_code].Latitude)
    df_toronto_g.Longitude[i] = float(df_coordinates[df_coordinates['Postal Code'] == post_code].Longitude)

In [34]:
df_toronto_g.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


In [36]:
df_toronto_g[df_toronto_g.Postcode == "M5A"]

,Postcode,Borough,Neighbourhood,Latitude,Longitude
53,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6543,-79.3606
